In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

D:\Anaconda3\envs\opensource\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# 데이터 불러오기
data = pd.read_csv('./kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
data.shape

(21613, 14)

# 의미가 없다고 판단되는 변수 제거

In [4]:
new_data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1, inplace = False)

# 범주형 변수를 이진형 변수로 변환
    - 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기때문에, 0 또는 1로 표현한다.
    - 데이터에서 0, 1로 이미 표현하고 있으므로, 과정 생략

# 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [13]:
feature_columns = list(new_data.columns.difference(['price']))
X = new_data[feature_columns]
y = new_data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(15129, 8) (15129,) (6484, 8) (6484,)


# 학습 데이터를 선형 회귀 모형에 적합 후, 평가 데이터로 검증(Stats_Models)

In [14]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant='add')
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Wed, 05 Aug 2020   Prob (F-statistic):               0.00
Time:                        10:30:12   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
sm_test_x = sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)

In [16]:
sqrt(mean_squared_error(sm_model_predict, test_y))  ## RMSE

239804.2967085816

# Bagging 결과가 일반적인 결과보다 좋은지 확인

In [21]:
import random

bagging_predict_result = list()
data_index = [data_index for data_index in range(train_x.shape[0])]
for _ in range(10):
    random_data_index = np.random.choice(data_index, train_x.shape[0])  # 복원 추출
    sm_train_x = train_x.iloc[random_data_index]
    sm_train_y = train_y.iloc[random_data_index]
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    pred = fitted_sm_model.predict(sm_test_x)
    
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred, test_y)))

240165.0765037411
240606.0317633463
239949.44603251392
240106.78100375814
240150.3070327486
240681.14151689946
239678.6668500614
239615.68833267424
239857.1238186125
240962.79906907608


In [22]:
bagging_predict = list()
for idx in range(test_x.shape[0]):
    temp_predict = list()
    for idx2 in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[idx2].values[idx])
    bagging_predict.append(np.mean(temp_predict))

In [24]:
bagging_predict

[563295.7952489977,
 710235.7844003903,
 1119918.8577716486,
 1477213.4379097559,
 700138.418165599,
 385208.9231830874,
 787338.9859921717,
 484246.04752681206,
 499781.89194658946,
 537192.8439958385,
 642527.2580470731,
 408957.6848560533,
 268051.0948871834,
 280015.47925554577,
 335903.3227559156,
 1264012.84870525,
 321426.9913899378,
 1034899.8675551492,
 260524.81872152007,
 603296.5596268853,
 390062.46812302107,
 1302832.6216493184,
 822190.9778887549,
 581892.8958959831,
 596502.553831863,
 570184.344569287,
 265439.32361902454,
 40725.667626986804,
 563919.8338994347,
 639869.0195463194,
 563826.0949042061,
 458223.1064397752,
 553962.9556190476,
 685283.4210006479,
 409204.5440777285,
 876063.0445565674,
 938294.0685878502,
 637853.2833479298,
 390418.696256131,
 1081092.2051696586,
 452596.3632494652,
 148658.08348310596,
 488898.10392060364,
 220731.08224263476,
 65082.797400014926,
 -45461.38972426321,
 247325.29149878328,
 283499.24495287926,
 362455.25367983663,
 7182

In [25]:
sqrt(mean_squared_error(bagging_predict, test_y))

239962.65112485457

# 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증(Scikit-Learn)

In [26]:
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()
linear_model1 = regression_model.fit(train_x, train_y)


# Bagging을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)|

In [29]:
from sklearn.ensemble import BaggingRegressor

bagging_model = BaggingRegressor(base_estimator=regression_model, n_estimators=10)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

239668.86785582433


In [28]:
bagging_model = BaggingRegressor(base_estimator=regression_model, n_estimators=30)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

239920.0373064702


# 학습 데이터를 Decision Tree 모형에 적합 후 평가 데이터로 검증

In [30]:
from sklearn.tree import DecisionTreeRegressor

decision_tree_model = DecisionTreeRegressor()
tree_model = decision_tree_model.fit(train_x, train_y)
predict_tree = tree_model.predict(test_x)

print(sqrt(mean_squared_error(predict_tree, test_y)))

291728.71160249365


In [33]:
bagging_predict_result = list()
data_index = [data_index for data_index in range(train_x.shape[0])]
for _ in range(10):
    random_data_index = np.random.choice(data_index, train_x.shape[0])  # 복원 추출
    sm_train_x = train_x.iloc[random_data_index]
    sm_train_y = train_y.iloc[random_data_index]
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    decision_tree_model = DecisionTreeRegressor()
    tree_model = decision_tree_model.fit(sm_train_x, sm_train_y)
    predict_tree = tree_model.predict(sm_test_x)
    
    bagging_predict_result.append(predict_tree)
    print(sqrt(mean_squared_error(predict_tree, test_y)))

306090.12992674543
298743.1214172224
277504.57923666044
292746.04095271684
293237.75097924354
288425.319838821
296009.0753059393
287241.3302135622
288768.124943108
281975.7849077876


In [34]:
bagging_predict = list()
for idx in range(test_x.shape[0]):
    temp_predict = list()
    for idx2 in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[idx2][idx])
    bagging_predict.append(np.mean(temp_predict))

In [35]:
bagging_predict

[345390.0,
 713500.0,
 930850.0,
 1469000.0,
 589057.1421474359,
 330100.0,
 865417.5,
 432633.3333333333,
 465372.912719533,
 450626.13095238095,
 625230.0,
 350805.9523809524,
 305133.3333333333,
 372205.998015873,
 425135.0,
 1284138.3333333333,
 437776.4304093568,
 1101600.0,
 251592.32380952378,
 645283.4880952381,
 339840.0,
 2344000.0,
 535328.3333333333,
 408574.126984127,
 557293.3333333334,
 533866.6666666666,
 275314.2397186147,
 316056.6666666666,
 391434.052631579,
 629195.0,
 590485.0,
 510447.3333333333,
 549185.3333333334,
 462845.0,
 371200.00000000006,
 1030200.0,
 1034900.0,
 625000.0,
 455183.0,
 1786000.0,
 452940.5,
 283095.0,
 496808.4527777778,
 261170.7,
 260396.66666666666,
 202235.0,
 282915.0,
 292526.325285138,
 321146.5,
 553685.0,
 377737.79564879567,
 339880.0,
 789388.8,
 331837.6666666666,
 468304.3063613179,
 2066666.6666666667,
 452300.0,
 706283.3333333334,
 321541.6666666667,
 649405.5237153198,
 578819.5833333334,
 387192.5,
 201858.16666666666,
 

In [36]:
sqrt(mean_squared_error(bagging_predict, test_y))

239090.376109249

# Bagging을 이용하여 Decision Tree 모형에 적합 후 평가 (Sampling 10번)

In [40]:
from sklearn.ensemble import BaggingRegressor

bagging_model2 = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=10)
decision_tree_model2 = bagging_model2.fit(train_x, train_y)
predict2 = decision_tree_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

235789.41032586055


In [41]:
bagging_model2 = BaggingRegressor(base_estimator=decision_tree_model, n_estimators=30)
decision_tree_model2 = bagging_model2.fit(train_x, train_y)
predict2 = decision_tree_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

233967.06766149274
